In [2]:
using JSON, BenchmarkTools
using NetPricing, JuMP, Gurobi

# Import a problem from a file
file = "../tmp/000000-000000-g40-05-P.json"
#file = "../data/from_github/problems/paper/g40-05.json"
# file = "../tmp/000001-000000-000001-g40-05-P.json"


result = JSON.parsefile("../tmp/000001-000000-000001-g40-05-R.json");
trans = JSON.parsefile("../tmp/000001-000000-000001-g40-05-T.json");
vtrans =  trans["V"];
etrans =  trans["A"];

#prob = read_problem(file)

# Preprocess the problem for each commodity
#pprobs = preprocess(prob, maxpaths = 1000)

# Create a model
#model, forms = std_model(pprobs)
# model, forms = pastd_model(pprobs)
# model, forms = vf_model(pprobs)
# model, forms = pvf_model(pprobs)

# Add strong bilevel feasibility (optional, only available for pastd and pvf models)
#add_strong_bf_cuts(model, forms, maxpairs=10000, commpairs=100)

# Solve the model
#optimize!(model)

# Extract the result
#tvals = value.(model[:t]);                   # The prices t

Precompiling NetPricing
  ✓ NetPricing
  1 dependency successfully precompiled in 6 seconds. 102 already precompiled.


LoadError: Expected end of input
Line: 0
Around: ...h":true}][{"id":"000001-000000...
                    ^


In [2]:
Amap = Dict()
Vmap = Dict()
λvals = Dict()
xvals = Dict()
b = Dict()

primal_obj = Dict()
dual_obj = Dict()
prob_tmp = Dict()

for k in 1:length(forms)
    primal_repr = primal(forms[k])              # Primal representation
    dual_repr = NetPricing.dual(forms[k])       # Dual representation
    prob_k = problem(primal_repr)               # Preprocessed problem of forms[k]
    
    real_k = NetPricing.index(prob_k)
    prob_tmp[real_k] = prob_k
    
    primal_obj[real_k] = value(primal_repr.primalobj)   # Primal objective: c' x[k]
    dual_obj[real_k] = value(dual_repr.dualobj)         # Dual objective: b' λ[k]
        
    real_k = NetPricing.index(prob_k)
    Amap[real_k] = used_arcs(prob_k)		    		# List of edge index of the solution path
    Vmap[real_k] = used_nodes(prob_k)
    λvals[real_k] = value.(dual_repr.λ)                 # Dual prices λ[k] (only for dual-arc)
    xvals[real_k] = value.(primal_repr.x)
    b[real_k] = NetPricing.sourcesink_vector(prob_k)    # vector b source sink
end

In [3]:
k=1
real_k = NetPricing.index(problem(primal(forms[k])))
origin = NetPricing.orig(prob_tmp[real_k])
destination = NetPricing.dest(prob_tmp[real_k])

op1 = [key for (key, value) in vtrans if value == Vmap[real_k][origin]]
op2 = [key for (key, value) in vtrans if value == Vmap[real_k][destination]]
println("Problem info")
println(prob)

println()
println("USER k=",real_k)
println("Reduce problem \t\t\t\t\t\t|Transformed space\t|Original space")
println("origin\t\t\t\t\t|\t",origin,"\t|", Vmap[real_k][origin],"\t\t\t|", op1)
println("destination\t\t\t\t|\t",destination,"\t|", Vmap[real_k][destination],"\t\t\t|", op2)
println()
println("Dual objective b' λ[k]\t\t\t|\t", dual_obj[real_k])
println("Primal objective c' x[k]\t\t|\t", primal_obj[real_k])
println("Profit from user b' λ[k] - c' x[k]\t|\t", dual_obj[real_k]-primal_obj[real_k])
println("-----------------------------------------------------------\n")


println("tvals")
println("length\t\t\t\t\t|\t", length(tvals))
println("vector\t\t\t\t\t|\t", tvals.data)
println("index\t\t\t\t\t|\t",  axes(tvals, 1))
println()
println("-----------------------------------------------------------\n")


println("λvals")
println("length\t\t\t\t\t|\t", length(λvals[real_k]))
println("vector\t\t\t\t\t|\t", λvals[real_k])
println()
println("Vertices used index Vmap")
println("length\t\t\t\t\t|\t", length(Vmap[real_k]))
println("vector\t\t\t\t\t|\t", Vmap[real_k])
println("-----------------------------------------------------------\n")

println("Arcs used xvals")
println("length\t\t\t\t\t|\t", length(xvals[real_k]))
println("vector\t\t\t\t\t|\t", xvals[real_k])
println()
println("Arc index Amap")
println("length\t\t\t\t\t|\t", length(Amap[real_k]))
println("vector\t\t\t\t\t|\t", Amap[real_k])
println()
println("Arcs used index")
println("Amap[xvals .== 1]\t\t\t|\t" , Amap[real_k][xvals[real_k] .== 1.0])

println("-----------------------------------------------------------\n")


Problem info
Problem with {60 nodes, 206 arcs (42 tolled), 39 commodities}

USER k=1
Reduce problem 						|Transformed space	|Original space
origin					|	1	|2			|["2"]
destination				|	7	|14			|["14"]

Dual objective b' λ[k]			|	98.0
Primal objective c' x[k]		|	98.0
Profit from user b' λ[k] - c' x[k]	|	0.0
-----------------------------------------------------------

tvals
length					|	42
vector					|	[12.0, 34.0, 6.0, 58.0, 0.0, 38.0, 0.0, 36.0, 0.0, 83.0, 36.0, 59.0, 0.0, 12.0, 0.0, 0.0, 19.0, 4.0, 2.0, 41.0, 27.0, 23.0, 37.0, 0.0, 115.0, 49.0, 22.0, 58.0, 29.0, 0.0, 6.0, 43.0, 7.0, 10.0, 60.0, 15.0, 10.0, 53.0, 60.0, 51.0, 5.0, 27.0]
index					|	[24, 31, 39, 41, 42, 44, 47, 55, 56, 57, 58, 71, 74, 85, 89, 92, 101, 103, 105, 106, 107, 108, 110, 119, 122, 125, 128, 139, 143, 146, 150, 155, 161, 163, 164, 165, 166, 179, 181, 184, 195, 197]

-----------------------------------------------------------

λvals
length					|	7
vector					|	[98.0, 86.0, 52.0, 34.0, 78.44727979895896, 46.447279

In [4]:
# Commentaire 
# les problemes forms[k] pour k=3 retourne celui pour k=5 (il y a suppression de problemes)


In [4]:
## Functions
function expand_b(Vmap, nv, b)
    # b is the vector to expand in the sparse space of dimension nv
    # Vmap is the index mapping to place element of b in the vector of dimension nv
    bfull = Vector{AffExpr}(zeros(nv))
    bfull[Vmap] .= b
    return bfull
end

function projection(transformation::Dict, Nn)
    NS = [[] for _ in 1:maximum(values(transformation))]
    for i in keys(transformation)
    	if typeof(i)!=typeof(1)
    		j = parse(Int, i)
    	else
    		j=i
    	end
        append!(
                NS[transformation[i]], Nn[j]#N[parse(Int, i)]
        )
    end
    
    NT_min = [value(minimum(vec)) for vec in NS]
    NT_avg = [value(mean(vec)) for vec in NS]
    NT_max = [value(maximum(vec)) for vec in NS]
    return NT_min, NT_avg, NT_max
end
function retroprojection(transformation::Dict, NT)
    # transformation is a dict of the for "i":j
    Nn = zeros(length(transformation))
    for (k, v) in transformation
    	if typeof(k)!=typeof(1)
    		j = parse(Int, k)
    	else
    		j=k
    	end
        Nn[j] = NT[v]
    end
    
    return Nn
end

retroprojection (generic function with 1 method)

In [5]:
result[1]

Dict{String, Any} with 12 entries:
  "preprocess_time" => 1.29117
  "b"               => Dict{String, Any}("24"=>Any[0, -1, 0, 0, 0, 0, 0, 0, 0, …
  "Vmap"            => Dict{String, Any}("24"=>Any[7, 8, 9, 11, 12, 13, 14, 15,…
  "Amap"            => Dict{String, Any}("24"=>Any[19, 26, 33, 35, 37, 39, 40, …
  "id"              => "000001-000000-000001-g40-05-P-zip-x"
  "flow"            => Dict{String, Any}("24"=>Any[39, 57, 75, 93, 105, 109, 11…
  "xvals"           => Dict{String, Any}("24"=>Any[0.0, 0.0, 0.0, 0.0, 0.0, 1.0…
  "finish"          => true
  "obj_value"       => 67585.0
  "λvals"           => Dict{String, Any}("24"=>Any[34.0, 0.0, 18.0, 73.0, 42.0,…
  "tvals"           => Any[25.0, 34.0, 6.0, 58.0, 0.0, 38.0, 0.0, 42.0, 0.0, 83…
  "solve_time"      => 8.58935

([6.0, 35.0, 6.0, 35.0, 12.0, 35.0, 23.0, 27.0, 23.0, 20.0  …  14.0, 13.0, 35.0, 13.0, 13.0, 35.0, 13.0, 25.0, 35.0, 25.0], [6.0, 35.0, 6.0, 35.0, 12.0, 35.0, 23.0, 27.0, 23.0, 20.0  …  14.0, 13.0, 35.0, 13.0, 13.0, 35.0, 13.0, 25.0, 35.0, 25.0], [6.0, 35.0, 6.0, 35.0, 12.0, 35.0, 23.0, 27.0, 23.0, 20.0  …  14.0, 13.0, 35.0, 13.0, 13.0, 35.0, 13.0, 25.0, 35.0, 25.0])

In [5]:
# γ(A)' * λ~ <= γ(c) + γ(t) need linearize_commodity_primal_custom

# (c + t)' * x <= b' * γ^-1(λ~) need linearize_commodity_primal_custom

# (γ(c) + γ(t))' * x~ <= γ(b)' * λ~ need linearize_commodity_primal_custom

In [ ]:
# Il faut expand les vecteurs dans les dimensions originales

# Exemple pour lambda
nv = nodes(prob)
fullλ = expand_b(Vmap[real_k], nv, λvals[real_k])

for o in Vmap[real_k]
    println(o, "\t:",[key for (key, value) in vtrans if value == o])
end
retrofullλ = retroprojection(vtrans, value.(fullλ))

println("XXXXXXXXXXXXXX")


# Exemple pour tvals
na = length(arcs(prob))
amap = axes(tvals, 1)
for o in amap
    println(o, "\t:",[key for (key, value) in etrans if value == o])
end
fullt = expand_b(amap, na, tvals.data)
# println(value.(fullt))
tt = value.(fullt)
indice = findall(tt -> tt > 0, tt)
println(indice)

retrofullt = retroprojection(etrans, value.(fullt))
exl = expand_b(λvals, Vmap, 54);
bxl = expand_b(b, Vmap, 54);
bxl' * exl;
# println(bxl);

rb = retroprojectionN(vtrans, bxl);
println(findfirst(x -> x == 1, rb))
println(findfirst(x -> x == -1, rb))


# b_min, b_avg,b_max = projection_b(vtrans, bxl);
# println(b_min)
# r = b_min' * ones(length(b_min))
# r

using JuMP
using Gurobi
import JuMP.Containers: DenseAxisArray


# Define a JuMP model
model = Model(Gurobi.Optimizer)

# Define some variables and lists
ra = [[1], [2, 3], [4, 6], [5], [7]] # classe
etrans = Dict("1" => 1, "2" => 2, "3" => 2, "4" => 3, "5" => 4, "6" => 3, "7" => 5)

# Define the index sets and data
N = [10, 9, 8, 7, 6, 7]


a1 = [2, 3, 4, 6, 7]                              # indice des arcs controllés 
a1dict = Dict(a => i for (i, a) in enumerate(a1)) # mapping entre indice et indice réduit

# Variable t
@variable(model, 0 ≤ t[a=a1], upper_bound = N[a1dict[a]])





c = cost_vector(prob)
A = incidence_matrix(prob)
b = sourcesink_vector(prob)

# γ(A)' * λ~ <= γ(c) + γ(t) need linearize_commodity_primal_custom

# (c + t)' * x <= b' * γ^-1(λ~) need linearize_commodity_primal_custom

# (γ(c) + γ(t))' * x~ <= γ(b)' * λ~ need linearize_commodity_primal_custom

# A faire une seul fois
γa1 = []         # les indices des arcs controllés dans l'espace transformé
γa1dict = Dict() # mapping entre indices et classes d'équivalences d'indice
for e in ra
    if !isempty(intersect(e, a1))
        i = etrans[string(e[1])]
        append!(λa1, i)
        γa1dict[i] = e
    end
end

# variable artificiel γt
@variable(model, γt[a=λa1]) # pas besoin de borner voir les contraintes
for (k,v) in γa1dict
    @constraint(model, γt[k] == mean(t[v]))
end


@objective(model, Max, γt' * [1,-0.5, 2])# objective remains the same

# Print the model
optimize!(model)
println(value.(model[:t])) 

# println(value.(model[:λt])) 

In [2]:
t = [1,2,3]
t[[1,2]]

2-element Vector{Int64}:
 1
 2

In [4]:


intersect((2,3), [1,])

Int64[]

In [3]:
b = [1,2]
Vmap = [1,3]
bfull = Vector{AffExpr}(zeros(3))
bfull[Vmap] .= b
bfull[b]

2-element Vector{AffExpr}:
 1
 0

1
2
3
4
5
6
7
8
9
10
